# Fix simulation sier results
7 genes' sier PIP are greater than 1 since the summation of whole genome varbvs PIPs are greater than 1.

In [1]:
import pandas as pd, numpy as np
import os
from collections import Counter
cwd = os.path.expanduser("/home/min/GIT/cnv-gene-mapping/output")
name = "simu_dele_30_0610"
prefix = f"deletion.X.colnames_b30.{name}"

In [2]:
index = pd.read_csv(f"{cwd}/{name}/{prefix}.X.block_index.csv", sep = "\t", header = None, names = ["start", "end"])

In [3]:
index.shape

(565, 2)

For some big blocks, sier consumes more than 3 hours, resulting in no outcome in [sier_3], so there is no overall outcome for all blocks. Use MCMC result instead temporarily and combine sier results in all blocks with workflow command.

In [4]:
## to be added

## Due to workflow error, sier result per block does not multiply sum(varbvs_wg_pips) over the block if the summation is greater than 1. Use the process below to export accurate sier PIPs by dividing wg_pips summation.

Since varbvs_pip changed with different seeds, the process below deals with original regional varbvs pips and current varbvs pips.

In [5]:
# j = 0; k = 0; l = 0; m = 0
# pips_allblocks = np.loadtxt(f"{cwd}/{name}/{prefix}.varbvs_wg.pip", usecols = 1)
# for i, item in index.iterrows():
#     sumpip_block = np.sum(np.loadtxt(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.varbvs_pip", usecols = 1))
#     sumpip_new = sum(pips_allblocks[item[0]:(item[1]+1)])
#     sier = pd.read_csv(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.sier.pip", sep = "\t", header = None, names = ["gene", "pip"])
#     sier.to_csv(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.sier_original.pip", sep = "\t", header = False, index = False)
#     if sumpip_block > 1:
#         if sumpip_new > 1:
#             j += 1
#             sier["pip"] = [x/sumpip_block for x in sier["pip"]]
#         else:
#             k += 1
#             sier["pip"] = [x/sumpip_block*sumpip_new for x in sier["pip"]]
#     else:
#         if sumpip_new < 1:
#             l += 1
#             sier["pip"] = [x/sumpip_block*sumpip_new for x in sier["pip"]]
#         else:
#             m += 1
#             sier["pip"] = [x/sumpip_block for x in sier["pip"]]
#     sier.to_csv(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.sier.pip", sep = "\t", header = False, index = False)

Since regional varbvs pips are not obtained in pipeline due to mistakes, sier results need to be multiplied by sum varbvs pips.

In [6]:
j = 0
for i, item in index.iterrows():
    sumpip_block = np.sum(np.loadtxt(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.varbvs_pip", usecols = 1))
    sier = pd.read_csv(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.sier.pip", sep = "\t", header = None, names = ["gene", "pip"])
    sier.to_csv(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.sier_original.pip", sep = "\t", header = False, index = False)
    if sumpip_block < 1:
        j += 1
        sier["pip"] = [x*sumpip_block for x in sier["pip"]]
        sier.to_csv(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.sier.pip", sep = "\t", header = False, index = False)

In [8]:
j

502

sier final output (pip.gz) is not generated. Use output in each blocks to combine.

In [4]:
res = pd.DataFrame(columns = ["gene", "pip"])
for i, item in index.iterrows():
    sier = pd.read_csv(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.sier.pip", sep = "\t", header = None, names = ["gene", "pip"])
    res = pd.concat([res, sier])

In [5]:
res = res.set_index([[i for i in range(res.shape[0])]])

In [6]:
res[res["gene"] == "NPIPA1"]

,gene,pip
2049,NPIPA1,1.585993e-16


In [7]:
res.to_csv(f"{cwd}/{name}/{prefix}.y.sier_pip.gz", compression = "gzip", sep = "\t", header = False, index = False)

In [8]:
res.head()

,gene,pip
0,TTLL10,0.125
1,TNFRSF18,0.125
2,TNFRSF4,0.125
3,SDF4,0.125
4,B3GALT6,0.125


Fit hybrid outcome: combine MCMC and sier results after determining blocked whole genome varbvs summation. If summation is greater than or equal to 1, use MCMC result, otherwise, use sier result.

In [4]:
res = pd.DataFrame(columns = ["gene", "pip"])
j = 0
for i, item in index.iterrows():
    sumpip = np.sum(np.loadtxt(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.varbvs_pip", usecols = 1))
    if sumpip >= 1:
        pips = pd.read_csv(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.mcmc.pip", sep = "\t", header = None, names = ["gene", "pip"])
        j += 1
    else:
        pips = pd.read_csv(f"{cwd}/{name}/{prefix}.X_block_{item[0]}_{item[1]}/block_{item[0]}_{item[1]}.sier.pip", sep = "\t", header = None, names = ["gene", "pip"])
    res = pd.concat([res, pips])

In [5]:
res.to_csv(f"{cwd}/{name}/{prefix}.y.hybrid_pip.gz", compression = "gzip", sep = "\t", header = False, index = False)

In [6]:
res.shape

(2756, 2)

In [7]:
res.head()

,gene,pip
0,TTLL10,0.125
1,TNFRSF18,0.125
2,TNFRSF4,0.125
3,SDF4,0.125
4,B3GALT6,0.125


In [8]:
j

63